In [4]:
import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog
import sqlite3

class Kitap:
    def __init__(self, baslik, yazar, yayin_evi, yayin_yili):
        self.baslik = baslik
        self.yazar = yazar
        self.yayin_evi = yayin_evi
        self.yayin_yili = yayin_yili

class Uye:
    def __init__(self, ad, soyad, uye_numarasi):
        self.ad = ad
        self.soyad = soyad
        self.uye_numarasi = uye_numarasi

class Kutuphane:
    def __init__(self):
        self.conn = sqlite3.connect('kutuphane.db')
        self.cursor = self.conn.cursor()
        self._create_tables()
        
    def _create_tables(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Kitaplar (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                baslik VARCHAR(300),
                yazar VARCHAR(100),
                yayin_evi VARCHAR(200),
                yayin_yili DATE
            )
        ''')
        
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Uyeler (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                ad VARCHAR(100),
                soyad VARCHAR(100),
                uye_numarasi VARCHAR(11)
            )
        ''')
        
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS OduncKitaplar (
                kitap_id INTEGER,
                uye_id INTEGER,
                FOREIGN KEY (kitap_id) REFERENCES Kitaplar (id),
                FOREIGN KEY (uye_id) REFERENCES Uyeler (id),
                PRIMARY KEY (kitap_id, uye_id)
            )
        ''')
        
        self.conn.commit()
        
    def _get_kitap_id(self, baslik):
        self.cursor.execute('SELECT id FROM Kitaplar WHERE baslik = ?', (baslik,))
        result = self.cursor.fetchone()
        return result[0] if result else None
    
    def _get_uye_id(self, uye_numarasi):
        self.cursor.execute('SELECT id FROM Uyeler WHERE uye_numarasi = ?', (uye_numarasi,))
        result = self.cursor.fetchone()
        return result[0] if result else None
    
    def kitap_ekle(self, kitap):
        self.cursor.execute('INSERT INTO Kitaplar (baslik, yazar, yayin_evi, yayin_yili) VALUES (?, ?, ?, ?)',
                            (kitap.baslik, kitap.yazar, kitap.yayin_evi, kitap.yayin_yili))
        self.conn.commit()
        
    def kitap_sil(self, baslik):
        kitap_id = self._get_kitap_id(baslik)
        if kitap_id:
            self.cursor.execute('DELETE FROM Kitaplar WHERE id = ?', (kitap_id,))
            self.conn.commit()
        
    def uye_ekle(self, uye):
        self.cursor.execute('INSERT INTO Uyeler (ad, soyad, uye_numarasi) VALUES (?, ?, ?)',
                            (uye.ad, uye.soyad, uye.uye_numarasi))
        self.conn.commit()
        
    def uye_sil(self, uye_numarasi):
        uye_id = self._get_uye_id(uye_numarasi)
        if uye_id:
            self.cursor.execute('DELETE FROM Uyeler WHERE id = ?', (uye_id,))
            self.conn.commit()
        
    def odunc_almak(self, baslik, uye_numarasi):
        kitap_id = self._get_kitap_id(baslik)
        uye_id = self._get_uye_id(uye_numarasi)
        if kitap_id and uye_id:
            self.cursor.execute('SELECT * FROM OduncKitaplar WHERE kitap_id = ?', (kitap_id,))
            odunc_durumu = self.cursor.fetchone()
            if not odunc_durumu:
                self.cursor.execute('INSERT INTO OduncKitaplar (kitap_id, uye_id) VALUES (?, ?)', (kitap_id, uye_id))
                self.conn.commit()
    
    def geri_vermek(self, baslik):
        kitap_id = self._get_kitap_id(baslik)
        if kitap_id:
            self.cursor.execute('SELECT * FROM OduncKitaplar WHERE kitap_id = ?', (kitap_id,))
            odunc_durumu = self.cursor.fetchone()
            if odunc_durumu:
                self.cursor.execute('DELETE FROM OduncKitaplar WHERE kitap_id = ?', (kitap_id,))
                self.conn.commit()
    
    def kitaplari_goster(self):
        self.cursor.execute('SELECT * FROM Kitaplar')
        kitaplar = self.cursor.fetchall()
        return kitaplar
            
    def uyeleri_goster(self):
        self.cursor.execute('SELECT * FROM Uyeler')
        uyeler = self.cursor.fetchall()
        return uyeler
            
    def close_connection(self):
        self.conn.close()

def kitap_ekle():
    baslik = simpledialog.askstring("Kitap Ekle", "Kitap başlığı:")
    yazar = simpledialog.askstring("Kitap Ekle", "Yazar:")
    yayin_evi = simpledialog.askstring("Kitap Ekle", "Yayın evi:")
    yayin_yili = simpledialog.askstring("Kitap Ekle", "Yayın yılı:")
    if baslik and yazar and yayin_evi and yayin_yili:
        kitap = Kitap(baslik, yazar, yayin_evi, yayin_yili)
        kutuphane.kitap_ekle(kitap)
        messagebox.showinfo("Başarılı", f"{baslik} kütüphanenize eklendi.")
    else:
        messagebox.showwarning("Hata", "Lütfen tüm bilgileri doldurun.")

def kitap_sil():
    baslik = simpledialog.askstring("Kitap Sil", "Silmek istediğiniz kitabın adını giriniz:")
    if baslik:
        kutuphane.kitap_sil(baslik)
        messagebox.showinfo("Başarılı", f"{baslik} kütüphanenizden silindi.")
    else:
        messagebox.showwarning("Hata", "Lütfen bir kitap adı giriniz.")

def uye_ekle():
    ad = simpledialog.askstring("Üye Ekle", "Üye adı:")
    soyad = simpledialog.askstring("Üye Ekle", "Üye soyadı:")
    uye_numarasi = simpledialog.askstring("Üye Ekle", "Üye numarası:")
    if ad and soyad and uye_numarasi:
        uye = Uye(ad, soyad, uye_numarasi)
        kutuphane.uye_ekle(uye)
        messagebox.showinfo("Başarılı", f"{ad} {soyad} kütüphanenize eklendi.")
    else:
        messagebox.showwarning("Hata", "Lütfen tüm bilgileri doldurun.")

def uye_sil():
    uye_numarasi = simpledialog.askstring("Üye Sil", "Silmek istediğiniz üyenin numarasını giriniz:")
    if uye_numarasi:
        kutuphane.uye_sil(uye_numarasi)
        messagebox.showinfo("Başarılı", f"{uye_numarasi} numaralı üye kütüphanenizden silindi.")
    else:
        messagebox.showwarning("Hata", "Lütfen bir üye numarası giriniz.")

def odunc_almak():
    baslik = simpledialog.askstring("Kitap Ödünç Al", "Kitabın adını giriniz:")
    uye_numarasi = simpledialog.askstring("Kitap Ödünç Al", "Üye numarasını giriniz:")
    if baslik and uye_numarasi:
        kutuphane.odunc_almak(baslik, uye_numarasi)
        messagebox.showinfo("Başarılı", f"{baslik} adlı kitap ödünç alındı.")
    else:
        messagebox.showwarning("Hata", "Lütfen tüm bilgileri doldurun.")

def geri_vermek():
    baslik = simpledialog.askstring("Kitap Geri Ver", "Kitabın adını giriniz:")
    if baslik:
        kutuphane.geri_vermek(baslik)
        messagebox.showinfo("Başarılı", f"{baslik} adlı kitap geri verildi.")
    else:
        messagebox.showwarning("Hata", "Lütfen bir kitap adı giriniz.")

def kitaplari_goster():
    kitaplar = kutuphane.kitaplari_goster()
    if not kitaplar:
        messagebox.showinfo("Kitaplar", "Henüz kütüphanenize eklenmiş bir kitap yok.")
    else:
        kitaplar_str = "\n".join([f"{kitap[1]} - {kitap[2]} ({kitap[3]})" for kitap in kitaplar])
        messagebox.showinfo("Kitaplar", kitaplar_str)

def uyeleri_goster():
    uyeler = kutuphane.uyeleri_goster()
    if not uyeler:
        messagebox.showinfo("Üyeler", "Henüz kütüphanenize eklenmiş bir üye yok.")
    else:
        uyeler_str = "\n".join([f"{uye[1]} {uye[2]} - {uye[3]}" for uye in uyeler])
        messagebox.showinfo("Üyeler", uyeler_str)

def cikis():
    kutuphane.close_connection()
    root.destroy()

kutuphane = Kutuphane()

root = tk.Tk()
root.title("Kütüphane Yönetim Sistemi")

# Menü butonları
menu_frame = tk.Frame(root)
menu_frame.pack(pady=20)

kitap_ekle_btn = tk.Button(menu_frame, text="Kitap Ekle", command=kitap_ekle)
kitap_ekle_btn.grid(row=0, column=0, padx=10, pady=10)

kitap_sil_btn = tk.Button(menu_frame, text="Kitap Sil", command=kitap_sil)
kitap_sil_btn.grid(row=0, column=1, padx=10, pady=10)

uye_ekle_btn = tk.Button(menu_frame, text="Üye Ekle", command=uye_ekle)
uye_ekle_btn.grid(row=1, column=0, padx=10, pady=10)

uye_sil_btn = tk.Button(menu_frame, text="Üye Sil", command=uye_sil)
uye_sil_btn.grid(row=1, column=1, padx=10, pady=10)

odunc_al_btn = tk.Button(menu_frame, text="Kitap Ödünç Al", command=odunc_almak)
odunc_al_btn.grid(row=2, column=0, padx=10, pady=10)

geri_ver_btn = tk.Button(menu_frame, text="Kitap Geri Ver", command=geri_vermek)
geri_ver_btn.grid(row=2, column=1, padx=10, pady=10)

kitaplari_goster_btn = tk.Button(menu_frame, text="Kitapları Göster", command=kitaplari_goster)
kitaplari_goster_btn.grid(row=3, column=0, padx=10, pady=10)

uyeleri_goster_btn = tk.Button(menu_frame, text="Üyeleri Göster", command=uyeleri_goster)
uyeleri_goster_btn.grid(row=3, column=1, padx=10, pady=10)

cikis_btn = tk.Button(menu_frame, text="Çıkış", command=cikis)
cikis_btn.grid(row=4, column=0, columnspan=2, pady=20)

root.mainloop()